<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Other Terms for the GRFFE Right-Hand Sides: $\tilde{S}_i$ source term, $A_i$ gauge term, and $\psi^6 \Phi$

## Author: Patrick Nelson
### Formatting improvements courtesy Brandon Clark

This notebook documents the simpler terms required for the RHSs used in the evolution equations in `GiRaFFE_NRPy`. The evolution equations in GRFFE are as follows:
\begin{align}
\partial_t \tilde{S}_i + \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) &= \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} \\
\partial_t A_i = \epsilon_{ijk} v^j B^k &- \partial_i (\alpha \Phi - \beta^j A_j) \\
\partial_t [\sqrt{\gamma} \Phi] + \partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) &= - \xi \alpha [\sqrt{\gamma} \Phi] \\
\end{align}
We have already handled two of these terms in separate modules because they required extra care to be taken. The flux term $\partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right)$ was covered in [this tutorial](Tutorial-GiRaFFE_NRPy-Stilde-flux.ipynb), and the no-gauge term $\epsilon_{ijk} v^j B^k$ was covered in [this tutorial](Tutorial-GiRaFFE_NRPy-Induction_Equation.ipynb). The remaining terms are simpler, and will all be handled here: the curvature term $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$, the gauge term $- \partial_i (\alpha \Phi - \beta^j A_j)$, and the right-hand side of $\partial_t [\sqrt{\gamma} \Phi]$. All of these terms can be handled with basic finite-differencing, at which NRPy+ excels. 

This module makes heavy use of the [GRHD](Tutorial-GRHD_Equations-Cartesian.ipynb) and [GRFFE](Tutorial-GRFFE_Equations-Cartesian.ipynb) modules, which are already validated against IllinoisGRMHD. These modules contain several functions that allow us to generate many of the expressions we need with only one line. 

**Notebook Status:** <font color=red><b> In progress</b></font>

**Validation Notes:** **TODO**

### NRPy+ Source Code for this module: 
* **TODO**


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#stilde_rhs): The $\tilde{S}_i$ curvature term
1. [Step 3](#a_rhs): The $A_i$ gauge term
1. [Step 4](#psi6phi_rhs): The $\sqrt{\gamma} \Phi$ RHS
1. [Step 5](#code_validation): Code Validation against `GiRaFFE_NRPy.Stilde_source_A_gauge_Phi_rhs` NRPy+ Module
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code just adds the root directory of `sys.path` and imports core NRPy+ functionality.

In [1]:
# Step 1: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs"

# Here, we'll import the GRHD and GRFFE modules. 
import GRHD.equations as GRHD
import GRFFE.equations as GRFFE


<a id='stilde_rhs'></a>

# Step 2: The $\tilde{S}_i$ curvature term \[Back to [top](#toc)\]
$$\label{stilde_rhs}$$

Our first concern will be the evolution of the Poynting flux,
$$
\partial_t \tilde{S}_i + \partial_j \left( \alpha \sqrt{\gamma} T^j_{{\rm EM} i} \right) = \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}.
$$
Here, $\alpha$ is the lapse function, $\beta^i$ is the shift vector, $\gamma_{ij}$ is the three-metric tensor, $g_{\mu\nu}$ is the four metric
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix},
$$
$T^{\mu \nu}_{\rm EM} = b^2 u^{\mu} u^{\nu} + \frac{b^2}{2} g^{\mu\nu} - b^\mu b^\nu$ is the electromagnetic stress-energy tensor, $u^\mu$ is the four-velocity, and $b^\mu$ is related to the magnetic field $B^i$ by 
\begin{align}
\sqrt{4\pi} b^0 = B^0_{\rm (u)} &= \frac{u_j B^j}{\alpha} \\
\sqrt{4\pi} b^i = B^i_{\rm (u)} &= \frac{B^i + (u_j B^j) u^i}{\alpha u^0}\\
\end{align}
Our concern is the source term $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$. We'll suppose the flux term has already been computed and that we need to add this contribution on, *without* overwriting existing data. To do this with NRPy+, we'll use `declarerank1()` instead of `zerorank1()`. This will let us start with write code of the form `StildeD0 = StildeD0 + source_term`. In that way, we can write
$$
\partial_t \tilde{S}_i \ += \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}
$$

In [2]:
def compute_StildeD_source_term(gammaDD,betaU,alpha,gammaDD_dD,betaU_dD,alpha_dD,sqrt4pi,ValenciavU,BU):
    # First, we'll use the GRHD module to generate the metric derivatives, four-velocity, and small b
    GRHD.u4U_in_terms_of_ValenciavU__rescale_ValenciavU_by_applying_speed_limit(alpha,betaU,gammaDD, ValenciavU)

    # Next sqrt(gamma)
    GRHD.compute_sqrtgammaDET(gammaDD)

    # Then compute g4DD_zerotimederiv_dD
    GRHD.compute_g4DD_zerotimederiv_dD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD)

    # small b
    GRFFE.compute_smallb4U(gammaDD,betaU,alpha, GRHD.u4U_ito_ValenciavU, BU, sqrt4pi)
    GRFFE.compute_smallbsquared(gammaDD, betaU, alpha, GRFFE.smallb4U)
    # Electromagnetic stress-energy tensor
    GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, GRFFE.smallb4U, GRFFE.smallbsquared,GRHD.u4U_ito_ValenciavU)

    # Add the source term to the RHS
    global Stilde_rhsD
    Stilde_rhsD = ixp.zerorank1(DIM=3)
    for i in range(3):
        for mu in range(4):
            for nu in range(4):
                # \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}
                Stilde_rhsD[i] += sp.Rational(1,2) * alpha * GRHD.sqrtgammaDET * \
                                  GRFFE.TEM4UU[mu][nu] * GRHD.g4DD_zerotimederiv_dD[mu][nu][i+1]


<a id='a_rhs'></a>

# Step 3: The $A_i$ gauge term \[Back to [top](#toc)\]
$$\label{a_rhs}$$

Next, we'll code the gauge term of $A_i$, expressed as 
$$
\partial_t A_i \ +=-\partial_i (\alpha \Phi - \beta^j A_j).
$$
We'll do this in two steps. In the first, we'll compute the parenthetical. Then, we'll declare a derivative of that term and add it to the $A_i$ RHS.

In [3]:
def compute_AD_gauge_term(gammaDD,betaU,alpha,psi6Phi,AD):
    GRHD.compute_sqrtgammaDET(gammaDD)
    Phi = psi6Phi/GRHD.sqrtgammaDET
    global AevolParen
    # \alpha \Phi - \beta^j A_j
    AevolParen = alpha * Phi
    for j in range(3):
        AevolParen += -betaU[j] * AD[j]

    # Take the gradient of the parenthetical and subtract it from the RHS
    AevolParen_dD = ixp.declarerank1("AevolParen_dD",DIM=3)
    global A_rhsD
    A_rhsD = ixp.declarerank1("A_rhsD",DIM=3)
    for i in range(3):
        A_rhsD[i] += -AevolParen_dD[i]

<a id='psi6phi_rhs'></a>

# Step 4: The $\sqrt{\gamma} \Phi$ RHS \[Back to [top](#toc)\]
$$\label{psi6phi_rhs}$$

Finally, we'll write the evolution equation for the scalar potential $\Phi$. Note, however, that this is stored as $\psi^6 \Phi$, where $\psi^6 = \sqrt{\gamma}$. Below is the evolution equation, rewritten so that the time derivative is isolated on the left-hand side:
$$
\partial_t [\sqrt{\gamma} \Phi] = -\partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) - \xi \alpha [\sqrt{\gamma} \Phi].
$$
This will again consist of two steps: we will build the parenthetical, then add the derivatives and other terms to the RHS.

In [4]:
def compute_psi6Phi_rhs(gammaDD,betaU,alpha,AD,psi6Phi,xi_damping):
    GRHD.compute_sqrtgammaDET(gammaDD)
    gammaUU,unusedgammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
    AU = ixp.zerorank1()
    # Raise the index on A in the usual way:
    for i in range(3):
        for j in range(3):
            AU[i] = gammaUU[i][j] * AD[j]
    
    global PhievolParenU
    PhievolParenU = ixp.zerorank1(DIM=3)
    
    for i in range(3):
        # \alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]
        PhievolParenU[j] += alpha*GRHD.sqrtgammaDET*AU[j] - betaU[j]*psi6Phi
    
    # Tell NRPy+ to take the derivative numerically:
    PhievolParenU_dD = ixp.declarerank2("PhievolParenU_dD","nosym",DIM=3)
    # -\partial_j (\alpha\sqrt{\gamma}A^j - \beta^j [\sqrt{\gamma} \Phi]) - \xi \alpha [\sqrt{\gamma} \Phi]
    PhievolParen_innerProduct = sp.sympify(0)
    # Divergence of the parenthetical term
    for j in range(3):
        PhievolParen_innerProduct += PhievolParenU_dD[j][j]
    # Combine the divergence and the damping term
    global psi6Phi_rhs
    psi6Phi_rhs = -PhievolParen_innerProduct - xi_damping * alpha * psi6Phi


<a id='code_validation'></a>

# Step 5: Code Validation against `GiRaFFE_NRPy.Stilde_source_A_gauge_Phi_rhs` NRPy+ Module \[Back to [top](#toc)\]
$$\label{code_validation}$$

Here, as a code validation check, we verify agreement in the SymPy expressions for the $\texttt{GiRaFFE}$ evolution equations and auxiliary quantities we intend to use between
1. this tutorial and 
2. the NRPy+ [GiRaFFE_NRPy.Stilde_source_A_gauge_Phi_rhs](../../edit/in_progress/GiRaFFE_NRPy/Stilde_source_A_gauge_Phi_rhs.py) module.


In [5]:
# First define magnetohydrodynamical quantities
ValenciavU = ixp.declarerank1("ValenciavU", DIM=3)
BU         = ixp.declarerank1("BU", DIM=3)
AD         = ixp.declarerank1("AD", DIM=3)
psi6Phi    = sp.symbols('psi6Phi', real=True)

# Then ADM quantities
gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
betaU   = ixp.declarerank1("betaU", DIM=3)
alpha   = sp.symbols('alpha', real=True)

# Then declare derivatives
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
betaU_dD   = ixp.declarerank2("betaU_dD"  ,"nosym",DIM=3)
alpha_dD   = ixp.declarerank1("alpha_dD"          ,DIM=3)

# Then numerical constant
sqrt4pi = sp.symbols('sqrt4pi', real=True)
xi_damping = sp.symbols('xi_damping', real=True)

# Import the module to test:
import GiRaFFE_NRPy.Stilde_source_A_gauge_Phi_rhs as rhs

all_passed=True
def comp_func(expr1,expr2,basename,prefixname2="C2P_P2C."):
    if str(expr1-expr2)!="0":
        print(basename+" - "+prefixname2+basename+" = "+ str(expr1-expr2))
        all_passed=False

def gfnm(basename,idx1,idx2=None,idx3=None):
    if idx2==None:
        return basename+"["+str(idx1)+"]"
    if idx3==None:
        return basename+"["+str(idx1)+"]["+str(idx2)+"]"
    return basename+"["+str(idx1)+"]["+str(idx2)+"]["+str(idx3)+"]"

expr_list = []
exprcheck_list = []
namecheck_list = []

compute_StildeD_source_term(gammaDD,betaU,alpha,gammaDD_dD,betaU_dD,alpha_dD,sqrt4pi,ValenciavU,BU)
compute_AD_gauge_term(gammaDD,betaU,alpha,psi6Phi,AD)
compute_psi6Phi_rhs(gammaDD,betaU,alpha,AD,psi6Phi,xi_damping)
rhs.compute_StildeD_source_term(gammaDD,betaU,alpha,gammaDD_dD,betaU_dD,alpha_dD,sqrt4pi,ValenciavU,BU)
rhs.compute_AD_gauge_term(gammaDD,betaU,alpha,psi6Phi,AD)
rhs.compute_psi6Phi_rhs(gammaDD,betaU,alpha,AD,psi6Phi,xi_damping)

namecheck_list.extend(["AevolParen","psi6Phi_rhs"])
exprcheck_list.extend([rhs.AevolParen,rhs.psi6Phi_rhs])
expr_list.extend([AevolParen,psi6Phi_rhs])
for i in range(3):
    namecheck_list.extend([gfnm("Stilde_rhsD",i),gfnm("A_rhsD",i),gfnm("PhievolParenU",i)])
    exprcheck_list.extend([rhs.Stilde_rhsD[i],rhs.A_rhsD[i],rhs.PhievolParenU[i]])
    expr_list.extend([Stilde_rhsD[i],A_rhsD[i],PhievolParenU[i]])

for i in range(len(expr_list)):
    comp_func(expr_list[i],exprcheck_list[i],namecheck_list[i])

import sys
if all_passed:
    print("ALL TESTS PASSED!")
else:
    print("ERROR: AT LEAST ONE TEST DID NOT PASS")
    sys.exit(1)


ALL TESTS PASSED!


<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.pdf](Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [6]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.ipynb
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.tex
!pdflatex -interaction=batchmode Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.tex
!rm -f Tut*.out Tut*.aux Tut*.log

This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
